<h1>Finite Difference Methods (FDM) Options Pricing</h1>

In this python notebook, we will be pricing exotic options by using FDM PDE solver.

<h2> 1.0 Importing Packages </h2>

In [1]:
from FDM.finite_difference_methods import *
from FDM.options_FDM import *

<h2> 2.0 Initialise Options </h2>

Code chunk below will be initialising the variables

In [6]:
# Initialising variables for the pde
spot_init = 110  # Initial Spot price S_0
strike    = 100  # Strike price
tau       = 6/12  # Time to maturity
r         = 0.05 # Risk-free rate
sigma     = lambda t, s: 0.2 # Function of volatility (in this case constant volatility)

dividend  = 0.05
bounds    = (0, 200)
option_type = 'call'

barrier = 118
knock   = 'up'


Code chunk below will be initialising the options PDE

In [7]:
options_pde = {'European': OptionsBlackScholes(strike, tau, r, sigma, dividend, bounds, option_type),
               'American': AmericanOptionsBlackScholes(strike, tau, r, sigma, dividend, bounds, option_type),
               f'Knock {knock} and out Barrier': KnockOutOptionsBlackScholes(strike, tau, r, sigma, dividend, barrier, knock, bounds, option_type)}

<h2> Compute price with FDM </h2>

The code chunk below will be defining a simple function for pricing options with a given PDE and selected scheme to solve the grid. This function will reduce the code necessary to compute price of different options

In [8]:
def compute_price_fdm(options_pde: OptionsBlackScholes, spot_init: float, PDESolverScheme: PDESolver, imax: int=1700, jmax: int=200, check_stable: bool=False) -> float:
    # Initialise PDE solver
    pde_solver = PDESolverScheme(options_pde, imax, jmax)

    # Check if it is stable
    if check_stable:
        pde_solver.check_stability(0, spot_init)

    # compute grid 
    pde_solver.solve_grid()

    return pde_solver.interpolate(0, spot_init)

<h3> Pricing with Explicit Scheme</h3>

In [9]:
print(f'Pricing with Explicit Scheme (Strike: {strike}, tau: {tau})')
for pde in options_pde:
    price = compute_price_fdm(options_pde[pde], spot_init, ExplicitScheme)
    print(f"{pde} {option_type} Option: price: {price}")

Pricing with Explicit Scheme (Strike: 100, tau: 0.5)
European call Option: price: 11.907756842548661
American call Option: price: 12.003228209939094
Knock up and out Barrier call Option: price: 1.1276648548763928


<h3> Pricing with Implicit Scheme</h3>

In [21]:
print(f'Pricing with Implicit Scheme (Strike: {strike}, tau: {tau})')
for pde in options_pde:
    price = compute_price_fdm(options_pde[pde], spot_init, ImplicitScheme)
    print(f"{pde} {option_type} Option: price: {price}")

Pricing with Implicit Scheme (Strike: 100, tau: 0.5)
European put Option: price: 2.1542217125884053
American put Option: price: 2.160667096589237
Knock up and out Barrier put Option: price: 1.874212191501466


<h3> Pricing with Implicit Scheme</h3>

In [22]:
print(f'Pricing with Crank Nicolson Scheme (Strike: {strike}, tau: {tau})')
for pde in options_pde:
    price = compute_price_fdm(options_pde[pde], spot_init, CrankNicolsonScheme)
    print(f"{pde} {option_type} Option: price: {price}")

Pricing with Crank Nicolson Scheme (Strike: 100, tau: 0.5)
European put Option: price: 3.188782273978385
American put Option: price: 3.205778647521699
Knock up and out Barrier put Option: price: 2.5033815239324224
